In [1]:
import pytorch_lightning as pl
import pandas as pd
import os 
import torch
import torchvision
from torch.utils.data import Dataset ,DataLoader
from torchmetrics import Accuracy
from pytorch_lightning.callbacks import ModelCheckpoint
import numpy as np
import torch
from sklearn.model_selection import train_test_split 
from torchvision import datasets, transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models
# from lightning.pytorch.loggers import WandbLogger
from pytorch_lightning.loggers import WandbLogger
import wandb




In [2]:
# Define data transformations
transform = transforms.Compose([
    transforms.Resize((256,256)),
#     transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset from directory
dataset = datasets.ImageFolder('/kaggle/input/inaturalist12k/Data/inaturalist_12K/train', transform=transform)
test_dataset = datasets.ImageFolder('/kaggle/input/inaturalist12k/Data/inaturalist_12K/val', transform=transform)



# Split dataset into training and testing sets
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [int(0.8*len(dataset)), len(dataset)-int(0.8*len(dataset))])

# Create data loader objects for training and testing sets
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=64, shuffle=False)

wandb_logger = WandbLogger(project="Assignment 2 Part 1 a")



# use directly wandb module
# wandb.config["key"] = "a2e6402ce9fe2ebe1f01d5332c4fafa210b0dc0c"
# wandb.config.update()

# Define the hyperparameters that you want to sweep over
hyperparameter_defaults = dict(
    dropout_rate=0.5,
    learning_rate=1e-3,
    batch_size=32
)

# Set up the configuration for your sweep using the `wandb.sweep` function
sweep_config = {
    'method': 'bayes', # or 'grid'
    'metric': {'name': 'val_loss', 'goal': 'minimize'},
    'parameters': {
        'dropout_rate': {'min': 0.1, 'max': 0.9},
        'learning_rate': {'min': 1e-5, 'max': 1e-1},
        'batch_size': {'values': [16, 32, 64]}
    }
}

# Initialize the WandB sweep
sweep_id = wandb.sweep(sweep_config)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: hpmtiu6u
Sweep URL: https://wandb.ai/cs22m008/uncategorized/sweeps/hpmtiu6u


CNN model 

In [3]:
class CNNModel(pl.LightningModule):

    def __init__(self):
        super(CNNModel, self).__init__()
#         cnn_out_channels = [16, 16, 16,32, 32]

        self.cnv1 = torch.nn.Conv2d(3, 64, 3)
        self.cnv2 = torch.nn.Conv2d(64, 64, 3)
        self.cnv3 = torch.nn.Conv2d(64, 64, 3)      
        self.cnv4 = torch.nn.Conv2d(64, 64, 3)
        self.cnv5 = torch.nn.Conv2d(64, 32, 3)
        self.activation_function=nn.Mish()
        self.bn = nn.BatchNorm2d(32)
        self.mxpool = nn.MaxPool2d(2)
        self.flat = nn.Flatten()
        self.fc= nn.Linear(1152,10)
        self.softmax = nn.Softmax()
        self.learning_rate=0.001
        self.dropout = nn.Dropout(p=0.3)
        self.save_hyperparameters()

    def forward(self,x):
#         print(x.shape)
        out=self.activation_function(self.cnv1(x))
        out=self.mxpool(out)
#         print("print: mp1    ",out.shape)

        out=self.activation_function(self.cnv2(out))
        out=self.mxpool(out)
#         print("print: mp2    ",out.shape)

        out=self.activation_function(self.cnv3(out))
        out=self.mxpool(out)
#         print("print: mp3    ",out.shape)

        out=self.activation_function(self.cnv4(out))
        out=self.mxpool(out)
#         print("print: mp4    ",out.shape)

        out=self.activation_function(self.cnv5(out))
        out=self.bn(out) ################################################################################################
        out = self.mxpool(out)
#         print("print: mp5    ",out.shape)
        out=self.flat(out)
#         print("print: result    ",out.shape)
        out = self.dropout(out)

        out = self.activation_function(self.fc(out))
        return out
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        self.log("train_loss", loss, prog_bar=True, on_epoch=True)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
        self.log("train_accuracy", accuracy, prog_bar=True,on_epoch=True)
        wandb.log({"train/loss": loss})

        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)

        self.log("val_loss", loss,prog_bar=True, on_epoch=True)
#         print("val Loss:",loss)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
#         print("val_accuracy: ",accuracy)
        self.log("val_accuracy", accuracy, prog_bar=True,on_epoch=True)

    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        accuracy = (logits.argmax(dim=1) == y).float().mean()
#         print("test_accuracy: ",accuracy)
        self.log("test_loss", loss, prog_bar=True, on_epoch=True)
        self.log("test_accuracy", accuracy, prog_bar=True,on_epoch=True)
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [4]:
# dataset = Dataset()
net = CNNModel()
trainer = pl.Trainer(max_epochs=4, accelerator="gpu", devices=1, logger=wandb_logger)
trainer.fit(model=net,train_dataloaders=train_dataloader,val_dataloaders=val_dataloader)
# trainer.test(model=net, datamodule=dataset, ckpt_path='best')

Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]